In [7]:
import paddle
import paddle.nn as nn
import numpy as np

# Generate sample data: y = 2x + 1 + noise
x = np.random.rand(100, 1).astype('float32')
y = 2 * x + 1 + np.random.normal(0, 0.05, x.shape).astype('float32')

# Define a simple linear model
class LinearNet(nn.Layer):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

# Initialize and train the model
model = LinearNet()
criterion = nn.MSELoss()
optimizer = paddle.optimizer.SGD(learning_rate=0.1, parameters=model.parameters())

for epoch in range(100):
    inputs = paddle.to_tensor(x)
    targets = paddle.to_tensor(y)
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    optimizer.clear_grad()

# Save model parameters
paddle.save(model.state_dict(), "/scratch/ling2/FSL-Mate/PaddleFSL/examples/molecular_property_prediction/app/linear_model.pdparams")
print("Model saved successfully.")


Model saved successfully.


In [10]:
import .shap

SyntaxError: invalid syntax (1529191561.py, line 1)

In [9]:
pip install visualdl

/bin/bash: /scratch/ling2/miniforge3/envs/paddleFSL1/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached visualdl-2.5.3-py3-none-any.whl.metadata (25 kB)
  Using cached flask_babel-4.0.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached rarfile-4.2-py3-none-any.whl.metadata (4.4 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.1.4-py3-none-any.whl.metadata (5.1 kB)
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
Using cached visualdl-2.5.3-py3-none-any.whl (6.3 MB)
Using cached flask_babel-4.0.0-py3-none-any.whl (9.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 43.9 MB/s eta 0:00:00:00:01
Using cached rarfile-4.2-py3-none-any.whl (29 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.9 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
import pandas as pd
import paddle
import paddle.nn as nn
import io
import base64

# Define the model (same structure as training)
class LinearNet(nn.Layer):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

# Load the trained model
model = LinearNet()
model.set_state_dict(paddle.load("linear_model.pdparams"))
model.eval()

# Create Dash app
app = dash.Dash(__name__)
app.title = "Paddle Inference Demo"

app.layout = html.Div([
    html.H2("Upload CSV for Prediction"),
    dcc.Upload(
        id='upload-data',
        children=html.Div(['Drag and drop or click to upload a CSV file (1 column of x values)']),
        style={
            'width': '100%', 'height': '60px', 'lineHeight': '60px',
            'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
            'textAlign': 'center', 'margin': '10px'
        },
        multiple=False
    ),
    html.Div(id='output')
])

@app.callback(
    Output('output', 'children'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def predict(contents, filename):
    if contents is None:
        return html.Div("Please upload a CSV file.")

    # Decode CSV content
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

    if df.shape[1] != 1:
        return html.Div("Please make sure the CSV contains exactly 1 column of numeric values.")

    # Inference
    input_tensor = paddle.to_tensor(df.values.astype('float32'))
    predictions = model(input_tensor).numpy().flatten()
    df['Prediction'] = predictions

    return html.Div([
        html.H5(f"Predictions from: {filename}"),
        dash_table.DataTable(
            data=df.to_dict('records'),
            columns=[{'name': col, 'id': col} for col in df.columns],
            style_table={'overflowX': 'auto'}
        )
    ])

if __name__ == '__main__':
    app.run(debug=True)
